# Coarse cell type annotation with Scyan

In scyan poetry env and after running
`os.chdir("../../Projects/HamburgCRC")`

In [ ]:
import os
import pandas as pd
import scyan
import anndata as ad

scyan_table_mini = pd.DataFrame({
"CD14": [-1,1,-1,-1,-1,-1],
"CD31": [-1,pd.NA,1,-1,-1,-1],
"CD45": [1,1,-1,-1,-1,-1],
"SMA": [-1,-1,-1,1,-1,-1],
"PanCK": [-1,-1,-1,-1,1,-1],
})

scyan_table_mini.index = ["Lympho", "Myelo", "Endo", "CAF", "Cancer", "Other"]

In [ ]:
scyan_table_mini.to_csv("scyan_table_mini.csv")
scyan_table_mini.drop("Other", inplace = True)
scyan_table_mini.to_csv("scyan_table_mini_noUnclear.csv")

In [ ]:
os.chdir("../../Projects/HamburgCRC")
scyan_table = pd.read_csv("scyan_table_mini.csv", index_col = 0)

cells = pd.read_csv("cell_table_transformed.csv", index_col=0)
# Exclude one FOV where most cells are badly segmented
cells = cells.loc[cells.fov != "C6h"]
lineage_channels = ['SMA','PanCK','CD45','CD31', "CD14"]

# Not sure why the indices were not matching in the first place
obs = cells.drop(lineage_channels, axis=1).reset_index(drop=True)
X = cells.loc[:,lineage_channels].reset_index(drop=True)

adata = ad.AnnData(X, obs=obs)

In [ ]:
scyan.preprocess.scale(adata)

# model = scyan.Scyan(adata, scyan_table, temperature = 1)
model = scyan.Scyan(adata, scyan_table, prior_std=0.35)
model.fit(min_delta = 0.5)
model.predict(log_prob_th=-20)

In [ ]:
scyan.data.add("scyan_run_mini_noUnclear", adata, scyan_table)